In [ ]:
import numpy as np
import json
import matplotlib.pyplot as plt
from src.simulation import get_spectra, generate_map
from pathlib import Path
from src.constants import *
from scipy.ndimage import convolve1d

path = Path('simulated_data')

In [ ]:
sps, calibration = get_spectra(20, ['Al', 'Fe'], save=True)

sps2 = convolve1d(sps, [1, 1, 1], axis=1, mode='nearest')
sps2 = sps2 + np.random.normal(0,sps.mean(),sps.shape)

In [ ]:
plt.plot(sps[5][:1000],color='blue')
plt.plot(sps2[5][:1000], color='orange')

In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import convolve2d

img = np.array(json.load(open(path / 'seed.json', 'r')), dtype=float)
print(np.unique(img))
plt.imshow(img, cmap='jet', aspect='auto')
plt.colorbar()

In [ ]:
kernel = np.asarray([
    [1, 1, 1],
    [1, 8, 1],
    [1, 1, 1],
], dtype=float)
kernel /= kernel.sum()

In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import convolve2d

zero_img = np.zeros(img.shape)
zero_img[img == 1] = 1.
zero_img[img != 1] = 0.

for i in range(100):
    zero_img = convolve2d(zero_img, kernel, mode='same', boundary='fill', fillvalue=.1)
plt.imshow(zero_img)

In [ ]:
one_img = np.zeros(img.shape)
one_img[img == 0] = 1.
one_img[img != 0] = 0.

for i in range(100):
    one_img = convolve2d(one_img, kernel, mode='same', boundary='fill', fillvalue=.1)
plt.imshow(one_img)

In [ ]:
result = zero_img / (zero_img + one_img)
result -= result.min()
result /= result.max()
result *= 20
result = np.rint(result)

plt.imshow(result, cmap='jet', aspect='auto')
plt.colorbar()

In [ ]:
y = np.zeros(result.shape) - 2

y[result < 4] = 0

y[(result >= 6) & (result < 12)] = 1

y[(result >= 16)] = 2

In [ ]:
plt.imshow(y)

In [ ]:
X = np.zeros(DIM + (calibration.shape[0],))
for i in range(sps.shape[0]):
    X[result == i, :] = sps[i]